In [1]:
import sys
import pickle
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import pandas
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics.pairwise import pairwise_distances_argmin 



Using TensorFlow backend.


In [2]:
MODULE_ROOT_NAME = "AgeEstimator"
MODULE_PATHS = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
MODULE_PATHS = list(
    filter(lambda x: x.endswith(MODULE_ROOT_NAME), MODULE_PATHS))
MODULE_PATH = MODULE_PATHS[0] if len(MODULE_PATHS) == 1 else ""
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)

In [3]:
from server.data.dataset import DataLoader

In [4]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",include_top=False)

def getPic(x_train):
    vgg16_feature_list = []
    for i in range(0, len(x_train)):
        pic = image.load_img(x_train[i], target_size=(224, 224))
        img_data = image.img_to_array(pic)
        
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())
    vgg16_feature_list_np = np.array(vgg16_feature_list)
    np.save("vgg16.npy",vgg16_feature_list_np)
    print("load image done")
    return vgg16_feature_list_np


In [5]:
def GNB_train(trainPoints, trainLabels):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB().fit(trainPoints, trainLabels)
    with open('gnb.pickle', 'wb') as f:
        pickle.dump(gnb, f)
    print("GNB done")

In [ ]:

if __name__ == '__main__':
    dl = DataLoader()
    use_bottleneck_features = True
    x_train, y_train = dl.load_train()
    x_test, y_test = dl.load_test()
    vgg16_feature_list_np = getPic(x_train)
    
    
    GNB_train(vgg16_feature_list_np,y_train)
